In this notebook, we create checklists that satisfy various fairness constraints on the UCI Adult dataset. Note that we use constraints that are quite loose due to time limitations. Using stricter constraints will require longer solution time, and could result in infeasibility.

In [2]:
import numpy as np
import pandas as pd

from IPChecklists.dataset import BinaryDataset

# if using CPLEX
from IPChecklists.model_cplex import ChecklistMIP
from IPChecklists.constraints_cplex import (MaxNumFeatureConstraint, MConstraint, FNRConstraint, GroupFNRConstraint,  
                                           GroupFNRGapConstraint, GroupFPRConstraint, GroupFPRGapConstraint)

# if using Python-MIP
# from IPChecklists.model_pythonmip import ChecklistMIP
# from IPChecklists.constraints_pythonmip import (MaxNumFeatureConstraint, MConstraint, FNRConstraint, GroupFNRConstraint, 
#                                                 GroupFNRGapConstraint, GroupFPRConstraint, GroupFPRGapConstraint)

Using CPLEX version 20.1.0.0


In [3]:
def get_datasets():
    target = 'target'
    pos_label = '>50K'
    
    df_train = pd.read_csv('./data/adult_train.csv', skipinitialspace = True)
    df_test = pd.read_csv('./data/adult_test.csv', skipinitialspace = True)
    df_test['target'] = df_test['target'].str.strip('.')
    
    for col in ['age', 'capital-gain', 'capital-loss', 'hours-per-week']: # continuous features
        df_train[col] = df_train[col].astype(float)
        df_test[col] = df_test[col].astype(float)

    features = ['age', 'workclass', 'education', 'marital-status',
               'occupation', 'relationship', 'capital-gain',
               'capital-loss', 'hours-per-week']

    df_train = df_train.sample(frac = 0.1, random_state = 42) # subsample for demonstration purposes for faster training
        
    train_ds = BinaryDataset(df_train, target_name = target, pos_label = pos_label, col_subset = features, 
                             add_complements=False, # don't add complements during binarization for faster training
                            protected_attrs = ['sex', 'race'])
    
    test_ds = train_ds.apply_transform(df_test)
    
    return train_ds, test_ds

### 1. Constraining the gap between groups

We create a checklist that minimizes 0-1 error (i.e. maximizes accuracy) with the following constraints:
- N <= 5
- M <= 3
- |FPR(Male) - FPR(Female)| <= 5%
- |FNR(Male) - FNR(Female)| <= 5%

In [4]:
train_ds, test_ds = get_datasets()

INFO:root:Removed 3 non-informative columns: {'capital-gain>=0.0', 'occupation==?', 'capital-loss>=0.0'}
INFO:root:Binary dataframe: 55 binary features and 3256 samples


In [5]:
model = ChecklistMIP(train_ds, cost_func = '01', compress = False) # need to set compress=False for fairness constraints  
model.add_constraint(GroupFNRGapConstraint('sex', eps = 0.05)) # FNR difference between all groups <= 5%
model.add_constraint(GroupFPRGapConstraint('sex', eps = 0.05)) # FNR difference between all groups <= 5%
model.build_problem(N_constraint = MaxNumFeatureConstraint('<=', 5), # N <= 5
                       M_constraint= MConstraint('<=', 3), # M <= 3
                     use_indicator = True # might result in better performance
                   ) 

In [6]:
stats = model.solve(max_seconds=60, display_progress=False) 

Found solution with objective 716.0001689378772 and optimality gap 48.74%.


In [7]:
check = model.to_checklist()
check

workclass==Federal-gov
education==Bachelors
marital-status==Married-civ-spouse
occupation==Exec-managerial
relationship==Unmarried

M = 2.0, N = 5.0

In [8]:
# training set performance
check.get_metrics(train_ds)

{'accuracy': 0.7800982800982801,
 'n_samples': 3256,
 'TN': 2204,
 'FN': 464,
 'TP': 336,
 'FP': 252,
 'error': 716,
 'TPR': 0.42,
 'FNR': 0.58,
 'FPR': 0.10260586319218241,
 'TNR': 0.8973941368078175,
 'precision': 0.5714285714285714,
 'pred_prevalence': 0.18058968058968058,
 'prevalence': 0.2457002457002457}

In [9]:
# training set fairness, note that the two constraints are satisfied
check.get_fairness_metrics(train_ds, attributes=['sex']) 

,Female,Male
accuracy,0.867619,0.738441
n_samples,1050.000000,2206.000000
TN,870.000000,1334.000000
FN,67.000000,397.000000
TP,41.000000,295.000000
FP,72.000000,180.000000
error,139.000000,577.000000
TPR,0.379630,0.426301
FNR,0.620370,0.573699
FPR,0.076433,0.118890


### 2. Constraining the per-group worst performance

We create a checklist that minimizes 0-1 error (i.e. maximizes accuracy) with the following constraints:
- N <= 8
- M <= 4
- max{FPR(Male), FPR(Female)} <= 35%
- max{FNR(Male), FNR(Female)} <= 35%

In [10]:
train_ds, test_ds = get_datasets()

INFO:root:Removed 3 non-informative columns: {'capital-gain>=0.0', 'occupation==?', 'capital-loss>=0.0'}
INFO:root:Binary dataframe: 55 binary features and 3256 samples


In [11]:
model = ChecklistMIP(train_ds, cost_func = '01', compress = False) # need to set compress=False for fairness constraints  
model.add_constraint(GroupFNRConstraint('sex', eps = 0.35)) 
model.add_constraint(GroupFPRConstraint('sex', eps = 0.35)) 
model.build_problem(N_constraint = MaxNumFeatureConstraint('<=', 8), # N <= 5
                       M_constraint= MConstraint('<=', 4), # M <= 3
                    use_indicator = True
                   ) 

In [12]:
stats = model.solve(max_seconds=60, display_progress=False) 

Advanced basis not built.


Found solution with objective 783.0002371974247 and optimality gap 56.23%.


In [13]:
check = model.to_checklist()
check

age>=37.0
workclass==Self-emp-not-inc
education==Prof-school
marital-status==Married-civ-spouse
occupation==Exec-managerial
relationship==Wife
hours-per-week>=40.0

M = 3.0, N = 7.0

In [14]:
# training set fairness, note that the two constraints are satisfied
check.get_fairness_metrics(train_ds, attributes=['sex']) 

,Female,Male
accuracy,0.851429,0.715775
n_samples,1050.000000,2206.000000
TN,822.000000,1052.000000
FN,36.000000,165.000000
TP,72.000000,527.000000
FP,120.000000,462.000000
error,156.000000,627.000000
TPR,0.666667,0.761561
FNR,0.333333,0.238439
FPR,0.127389,0.305152
